# Function Calling w/ JSON Schema

In [ ]:
from ollama import chat
from ollama import Client, ChatResponse, Tool

SEARCH = {
    "type": "function",
    "function": {
        "name": "search_facts",
        "description": "Search for data from the interest to help provide additional context to ground an answer.",
        "parameters": {
            "type": "object",
            "required": [
                "question"
            ],
            "properties": {
                "question": {
                    "type": "string",
                    "description": "The question to answer"
                }
            }
        },
    }
}
ADD = {
    "type": "function",
    "function": {
        "name": "add_two_numbers",
        "description": "Add two numbers",
        "parameters": {
            "type": "object",
            "required": [
                "a",
                "b"
            ],
            "properties": {
                "a": {
                    "type": "integer",
                    "description": "The first integer number"
                },
                "b": {
                    "type": "integer",
                    "description": "The second integer number"
                }
            }
        },
    }
}

TOOLS = [ADD, SEARCH]

FORMAT = {
    "anyOf": [
        {
            "type": "object",
            "properties": {
                "name": {
                    "type": "string",
                    "enum": [tool["function"]["name"]],
                },
                "parameters": tool["function"]["parameters"],
            },
            "required": [
                "name",
                "parameters", 
            ]
        }
        for tool in TOOLS
    ],
}

client = Client(host='https://ollama.k8s.mrv.thebends.org/')
response: ChatResponse = client.chat(
    model='llama3.1',
    messages=[
        {
            'role': 'user',
            'content': "write a report on the history of the internet",
        },
    ],
    tools=[Tool(**tool) for tool in TOOLS],
    format=FORMAT,
)

print("Content:")
print(response.message.content)

print("Tool calls:")
print(response.message.tool_calls)

Content:

Tool calls:
[ToolCall(function=Function(name='search_facts', arguments={'question': 'history of the internet'}))]
